In [1]:
import re
import csv
import math
import nltk
import copy
import string
from collections import Counter
from nltk.corpus import stopwords

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
nltk.download('stopwords')
stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
train_link='/content/drive/MyDrive/Exalens/BBC News Train.csv'

In [64]:
def data_loading(link):
    data=[]
    with open(link,mode= 'r') as file:
        csv_file= csv.reader(file)

        for lines in csv_file:
            data.append(lines)

    data.pop(0)
    for i in range(len(data)):
        data[i].pop(0)
  
    return data

In [65]:
def data_labels(data):
    labels=[]
    for i in range(len(data)):
        labels.append(data[i].pop(1))
    return labels

In [181]:
train_data= data_loading(train_link)
train_labels= data_labels(train_data)

In [67]:
def data_cleanup(text):
    l=[]
    fin=[]
    for word in text.split(' '):
        if word not in stops:
            l.append(word)
    x= ' '.join(l) 
    x= x.encode('ascii', 'ignore').decode()
    x= re.sub(r'https*\S+', ' ', x)
    x= re.sub(r'@\S+', ' ', x)
    x= re.sub(r'#\S+', ' ', x)
    x= re.sub(r'\'\w+', '', x)
    x= re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
    x= re.sub(r'\w*\d+\w*', '', x)
    x= re.sub(r'\s{2,}', ' ', x)
    fin.append(x)
    return fin

##Change this cell to len(train_data)

In [182]:
for i in range(len(train_data)):
    train_data[i]= data_cleanup(train_data[i][0])
    train_data[i]= train_data[i][0].split(' ')

In [183]:
uniq_labels=[]
for lab in train_labels:
    if lab not in uniq_labels:
        uniq_labels.append(lab)

In [184]:
data_list=[]
for u in uniq_labels:
    l_list=[]
    for i in range(len(train_data)):
        if train_labels[i]== u:
            l_list.append(train_data[i])
    data_list.append(l_list)
input_data= dict(zip(uniq_labels,data_list))   

In [73]:
test_link= '/content/drive/MyDrive/Exalens/BBC News Test.csv'

In [74]:
test_data= data_loading(test_link)

In [75]:
for i in range(len(test_data)):
    test_data[i]= data_cleanup(test_data[i][0])
    test_data[i]= test_data[i][0].split(' ')

In [214]:
class MultinomialNB:
    def __init__(self, articles_per_tag):
        # Don't change the following two lines of code.
        self.articles_per_tag = articles_per_tag  # See question prompt for details.
        self.train()
   
    def train(self):
        def get_unique_values(new_list):
            unique = []
            for ele in new_list:
                if ele not in unique:
                    unique.append(ele)
            return unique

        def word_counts(uni_wrds,docs):
            word_counter=[]
            for u in range(len(uni_wrds)):
                cnt=0
                for doc in range(len(docs)):
                    w_cnts= Counter(docs[doc])
                    cnt= cnt + w_cnts.get(uni_wrds[u],0)
                word_counter.append(cnt)
            return word_counter

        uniq_labels= self.articles_per_tag.keys()
        uniq_list=[]
        lab_wise_cnts=[]

        for ul in uniq_labels:
            lab_uniq_list=[]
            lab_data= self.articles_per_tag.get(ul)
            lc=0
            for d1 in range(len(lab_data)):
                word_list=get_unique_values(lab_data[d1])
                lab_uniq_list.extend(word_list)
                lc= lc+len(lab_data[d1])
            uniq_list.extend(lab_uniq_list)
            lab_wise_cnts.append(lc)
        uniq_list= get_unique_values(uniq_list) 
    
        lab_docword_counts=[]
        for doc_lab in uniq_labels:
            doc_list= self.articles_per_tag.get(doc_lab)
            dwc_dict= dict(zip(uniq_list,word_counts(uniq_list,doc_list)))
            lab_docword_counts.append(dwc_dict)
        
        self.train_wrd_cnt= dict(zip(uniq_labels,lab_docword_counts))
        self.train_lab_cnt= dict(zip(uniq_labels,lab_wise_cnts))      
        pass
    
    def predict(self, article):
        def probability(word, w_dict):
            prob= math.log((w_dict.get(word,0)+1)/(sum(w_dict.values())+2))  
            return prob

        uniq_labels= self.train_lab_cnt.keys()
        res=[]
        for d in range(len(article)):
            interim_res=[]
            for label in uniq_labels:
                p=0
                for k in range(len(article[d])):
                    p= p + probability(article[d][k],self.train_wrd_cnt.get(label))
                fin_p= round(p + math.log(self.train_lab_cnt.get(label)/sum(self.train_lab_cnt.values())),4)
            
                interim_res.append(fin_p)
            res.append(interim_res)
    
        results_dict=[]
        for r in range(len(res)):
            d= dict(zip(uniq_labels,res[r]))
            results_dict.append(d)

        return results_dict
        pass

In [215]:
mnb= MultinomialNB(input_data)
mnb.train()

In [216]:
new_test_results= mnb.predict(test_data)
print(len(new_test_results))

735


In [217]:
print(new_test_results[:5])

[{'business': -817.3246, 'tech': -843.3445, 'politics': -832.0478, 'sport': -746.1193, 'entertainment': -796.42}, {'business': -2120.0977, 'tech': -1934.0041, 'politics': -2136.9598, 'sport': -2283.8258, 'entertainment': -2215.7544}, {'business': -1301.0164, 'tech': -1316.4071, 'politics': -1281.1263, 'sport': -1063.794, 'entertainment': -1250.2868}, {'business': -1499.1667, 'tech': -1636.326, 'politics': -1585.5, 'sport': -1619.7699, 'entertainment': -1582.505}, {'business': -704.9955, 'tech': -729.4351, 'politics': -704.9155, 'sport': -623.2346, 'entertainment': -685.511}]


In [218]:
max_key=[]
for n in range(len(new_test_results)):
    max_key.append(max(new_test_results[n], key=lambda key: new_test_results[n][key]))

In [219]:
max_key[:10]

['sport',
 'tech',
 'sport',
 'business',
 'sport',
 'sport',
 'politics',
 'politics',
 'entertainment',
 'business']

In [152]:
samp_data={"politics":[['articles','writes','joel','furr','writes',],['distribution','world','following','posted']], "sports":[['article','writes','just','wanted'],['phillies','salvaged','their','weekend']], "tech":[['thanks','steve','your','helpful'],['please','unsubscribe','this','user']]}

In [ ]:
mnb= MultinomialNB(samp_data)
mnb.train()

In [163]:
art= ['article','writes','while','when','owned','plus','wanted','upgrade','memory','just','ordered','toolkit','from','macwarehouse','something','like','included','antistatic']

In [ ]:
my_work= mnb.predict(art)

In [197]:
def train_demo(articles_per_tag):
    def get_unique_values(new_list):
        unique = []
        for ele in new_list:
            if ele not in unique:
                unique.append(ele)
        return unique

    def word_counts(uni_wrds,docs):
        word_counter=[]
        for u in range(len(uni_wrds)):
            cnt=0
            for doc in range(len(docs)):
                w_cnts= Counter(docs[doc])
                cnt= cnt + w_cnts.get(uni_wrds[u],0)
            word_counter.append(cnt)
        return word_counter

    uniq_labels= articles_per_tag.keys()
    uniq_list=[]
    lab_wise_cnts=[]

    for ul in uniq_labels:
        lab_uniq_list=[]
        lab_data= articles_per_tag.get(ul)
        lc=0
        for d1 in range(len(lab_data)):
            word_list=get_unique_values(lab_data[d1])
            lab_uniq_list.extend(word_list)
            lc= lc+len(lab_data[d1])
        uniq_list.extend(lab_uniq_list)
        lab_wise_cnts.append(lc)
    uniq_list= get_unique_values(uniq_list) 
    
    lab_docword_counts=[]
    for doc_lab in uniq_labels:
        doc_list= articles_per_tag.get(doc_lab)
        dwc_dict= dict(zip(uniq_list,word_counts(uniq_list,doc_list)))
        lab_docword_counts.append(dwc_dict)
        
    train_wrd_cnt= dict(zip(uniq_labels,lab_docword_counts))
    train_lab_cnt= dict(zip(uniq_labels,lab_wise_cnts))

    return train_wrd_cnt, train_lab_cnt       

In [186]:
w_cnt,l_cnt= train_demo(input_data)

In [198]:
def predict_demo(data,results,labels):
    
    def probability(word, w_dict):
        prob= math.log((w_dict.get(word,0)+1)/(sum(w_dict.values())+2))  
        return prob

    uniq_labels= labels.keys()
    res=[]
    for d in range(len(data)):
        interim_res=[]
        for label in uniq_labels:
            p=0
            for k in range(len(data[d])):
                p= p + probability(data[d][k],results.get(label))
            
            fin_p= round(p + math.log(labels.get(label)/sum(labels.values())),4)
            
            interim_res.append(fin_p)
        res.append(interim_res)
    
    results_dict=[]
    for r in range(len(res)):
        d= dict(zip(uniq_labels,res[r]))
        results_dict.append(d)

    return results_dict

In [190]:
s_res= predict(test_data,w_cnt,l_cnt)

In [191]:
len(s_res)

735

In [ ]:
s_res[:5]

In [193]:
max_key=[]
for n in range(len(s_res)):
    max_key.append(max(s_res[n], key=lambda key: s_res[n][key]))

In [196]:
max_key[20:30]

['business',
 'politics',
 'sport',
 'business',
 'politics',
 'sport',
 'business',
 'sport',
 'sport',
 'business']